# PyKOALA Data Reduction Sequence (Science data)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import os
from astropy import units as u
from pykoala import __version__
import warnings

# You may want to comment the following line
warnings.filterwarnings("ignore")

# pyKOALA version
print("pyKOALA version: ", __version__)

In [ ]:
from pykoala.instruments.koala_ifu import koala_rss
from pykoala.corrections.atmospheric_corrections import AtmosphericExtCorrection, ADRCorrection
from pykoala.corrections.throughput import Throughput, ThroughputCorrection
from pykoala.cubing import CubeInterpolator, build_wcs
from pykoala.plotting.utils import qc_cube

atm_ext_corr = AtmosphericExtCorrection.from_text_file(
    AtmosphericExtCorrection.default_extinction)

### Throughput correction

In [ ]:
flat_rss = [koala_rss("data/combined_skyflat_red.fits")]
throughput_corr = ThroughputCorrection.from_rss(flat_rss, clear_nan=True, medfilt=10)

### Telluric correction

We will use the correction function derived during the standard stars reduction sequence.

In [ ]:
from pykoala.corrections.sky import TelluricCorrection

telluric_corr = TelluricCorrection.from_text_file('products/telluric_correction_red.dat')

## Load the science data

In [ ]:
sci_rss = []
aaomega_arms = {'blue': 1, 'red': 2}
# Choose which arm of the spectrograph is going to be used
arm = 'blue'
path_to_data = 'data'

adr_set = []

#for i in [31, 32, 33]:
for i in [34, 35, 36]:
    filename = f"data/27feb200{i}red.fits"
    rss = koala_rss(filename)

    rss = throughput_corr.apply(rss)
    rss = atm_ext_corr.apply(rss)
    rss = telluric_corr.apply(rss)
    # Telluric
    # Sky
    adr = ADRCorrection(max_adr=0.5*u.arcsec,
                        pol_deg=2, n_com_powers=4, clip_sigma=3.0)
    adr.estimate(rss, plot=False)
    rss = adr.apply(rss, copy=False)
    adr_set.append([rss.info["adr_offsets"]["dra_arcsec"],
                    rss.info["adr_offsets"]["ddec_arcsec"]])
    sci_rss.append(rss)

    rss.plot_rss_image(rss.snr, data_label="SNR/pixel")

## Astrometry Corrections and Registration

### Image Cross-correlation

The most sofisticated method to perform the registration of extended sources included in pyKOALA is based on the cross-correlation of two images.
### Centroid finding

A simple approach to find the offset between the different RSS is to find the center of light of the images (assuming that they contain the same sources).

In [ ]:
from pykoala.corrections.astrometry import AstrometryCorrection

astrom_corr = AstrometryCorrection()
offsets, fig = astrom_corr.register_crosscorr(sci_rss, qc_plot=True)
for offset in offsets:
    print("Offset (ra, dec) in arcsec: ", offset[0].to('arcsec'), offset[1].to('arcsec'))

In [ ]:
fig

For interpolating RSS data into a 3D datacube we will make use of the function *build_cube*. This method requires as input:
- A list of RSS objects. 
- The desired dimensions of the cube expressed as a 2-element tuple, corresponding to (ra, dec) in arcseconds.
- The pixel size of the cube in arcseconds.
- A list containing the ADR correction for every RSS (it can contain None) in the form: [(ADR_ra_1, ADR_dec_1), (ADR_ra_2, ADR_dec_2), (None, None)].
- Additional information to be included in *cube_info*

In [ ]:
datacube_shape = (sci_rss[0].wavelength.size, 40, 60)
ref_position = (sci_rss[0].wavelength[0],
                np.mean(sci_rss[0].info['fib_ra']),
                np.mean(sci_rss[0].info['fib_dec']))  # (deg, deg)
spatial_pixel_size = 1.0 * u.arcsec
spectral_pixel_size = sci_rss[0].wavelength[1] - sci_rss[0].wavelength[0]  # (angstrom)

print(f"Creating a WCS with\n position: {ref_position}\n Spatial pixel size: {spatial_pixel_size}\n Spectral pixel size: {spectral_pixel_size}")

wcs = build_wcs(datacube_shape=datacube_shape,
                reference_position=ref_position,
                spatial_pix_size=spatial_pixel_size,
                spectra_pix_size=spectral_pixel_size,
            )

interpolator = CubeInterpolator(rss_set=sci_rss, wcs=wcs, adr_set=adr_set,
                                kernel_scale=1.0,
                                cube_info=dict(
                                    name=rss.info['name'].split(' ')[0]))
cube = interpolator.build_cube()


## Sky substraction

In [ ]:
import pykoala.corrections.sky as sky
import importlib
importlib.reload(sky)

skymodel = sky.SkyFromObject(cube, bckgr_estimator='mad', source_mask_kappa_sigma=3, remove_cont=False)
skycorrection = sky.SkySubsCorrection(skymodel)
cube, _ = skycorrection.apply(cube)

In [ ]:
qc_cube(cube)

## Absolute flux calibration

In [ ]:
from pykoala.corrections.flux_calibration import FluxCalibration

fcal = FluxCalibration.from_text_file('products/response_HILT600_transfer_function.dat')

plt.figure()
plt.plot(fcal.response_wavelength, fcal.response)
plt.xlabel("Wavelength (AA)")
plt.ylabel("Response function (ADU/Flam)")
plt.show()

cube = fcal.apply(cube)

In [ ]:
fig = qc_cube(cube)
plt.show(plt.figure(fig))


In [ ]:
# Save the data cube
cube.to_fits("products/Henize_2-10_cube_no_sky.fits.gz", overwrite=True)